In [1]:
# Install and import the necessary libraries
!pip install -q torch
!pip install -q -U accelerate peft bitsandbytes transformers trl einops

In [2]:
import os
import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GPTNeoForCausalLM, 
    GPT2Tokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)

from trl import SFTTrainer

In [3]:
# Model
base_model = "EleutherAI/gpt-neo-2.7B"

# Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(base_model)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset, DatasetDict, Dataset
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a function to transform the dataset
def format_example(example):
    # Use the specified format for text
    example["text"] = f"### Instruction: {example['Question']} ### Assistant: {example['Answer']}"
    return example

train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset=train_test_split["train"]
train_dataset = train_dataset
test_dataset=train_test_split["test"]
test_dataset = test_dataset

# Apply the transformation to both the train and test datasets
train_dataset = train_dataset.map(format_example)

# Remove unnecessary columns and keep only the "text" column
train_dataset = train_dataset.remove_columns(["qtype", "Question", "Answer"])
# print(formatted_dataset)

# Preview the dataset
print(train_dataset)
print(test_dataset)

README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

Map:   0%|          | 0/13125 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 13125
})
Dataset({
    features: ['qtype', 'Question', 'Answer'],
    num_rows: 3282
})


In [5]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = GPTNeoForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map='auto',
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

In [6]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [7]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 100,
)

# LoRA configuration
peft_config = LoraConfig(
    r=64,                   #default=8
    lora_alpha= 16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "out_proj"] #["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ]
)


In [8]:
#print_trainable_parameters(model)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length= 512,
    tokenizer=tokenizer,
    args=training_arguments,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/13125 [00:00<?, ? examples/s]

In [9]:
# Train model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
100,2.124200
200,1.619700
300,1.539500
400,1.408200
500,1.349600
600,1.329300
700,1.295400
800,1.332900
900,1.343000
1000,1.265900


TrainOutput(global_step=6563, training_loss=1.2725923390862928, metrics={'train_runtime': 18368.6402, 'train_samples_per_second': 0.715, 'train_steps_per_second': 0.357, 'total_flos': 5.001239612802048e+16, 'train_loss': 1.2725923390862928, 'epoch': 1.0})

In [10]:
trainer.model.save_pretrained("./gptneo_finetuned_subjective/final_model")

In [11]:
from peft import PeftModel
f_model = PeftModel.from_pretrained(model,'/kaggle/working/gptneo_finetuned_subjective/final_model')
f_model = f_model.merge_and_unload()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [12]:
print(test_dataset[0])

{'qtype': 'frequency', 'Question': 'How many people are affected by microcephaly-capillary malformation syndrome ?', 'Answer': 'Microcephaly-capillary malformation syndrome is rare. About a dozen people have been diagnosed with the disorder.'}


In [14]:
from tqdm import tqdm

f_model.eval()  # Set the model to evaluation mode
predictions = []
all_preds = []  # Ensure this is defined for appending predictions
device = next(f_model.parameters()).device  # Ensure device compatibility
batch_size = 1  # Set a batch size if needed

for i, question in enumerate(tqdm(test_dataset, desc="Generating predictions", unit="question")):
    prompt = question['Question']

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = f_model.generate(inputs['input_ids'], max_length=70)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(response)

Generating predictions:   0%|          | 0/3282 [00:00<?, ?question/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Generating predictions:   0%|          | 1/3282 [00:03<3:30:32,  3.85s/question]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Generating predictions:   0%|          | 2/3282 [00:07<3:25:29,  3.76s/question]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Generating predictio

In [15]:
# print(predictions)
import csv

# Open the CSV file in write mode
with open("output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    
    # Optional: Write a header if needed
    writer.writerow(["Prediction"])

    # Write each item from the predictions list
    for item in predictions:
        writer.writerow([item])


In [16]:
import csv

# Open the CSV file in write mode
with open("true_output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    
    # Optional: Write a header if needed
    writer.writerow(["Prediction"])

    # Write each item from the predictions list
    for item in test_dataset['Answer']:
        writer.writerow([item])